In [ ]:
!pip install orbit-ml
import pandas as pd
import numpy as np
from math import pi
import matplotlib.pyplot as plt
import orbit
from orbit.models import KTR
from orbit.diagnostics.plot import plot_predicted_components
from orbit.utils.plot import get_orbit_style
from orbit.constants.palette import OrbitPalette

In [ ]:
%matplotlib inline
pd.set_option('display.float_format', lambda x: '%.5f' % x)
orbit_style = get_orbit_style()
plt.style.use(orbit_style);

In [ ]:
# Setting up Model Variables by transforming Data Columns

## Sorting by Date and other Date transformations
df_readability_kickstarter_small['project_launched_at'] = pd.to_datetime(df_readability_kickstarter_small['project_launched_at'])
df_readability_kickstarter_small.sort_values(by='project_launched_at', inplace=True)

df_readability_kickstarter_small['project_deadline'] = pd.to_datetime(df_readability_kickstarter_small['project_deadline'])
df_readability_kickstarter_small['timestamp_launched'] = df_readability_kickstarter_small['project_launched_at'].apply(lambda x: (datetime.timestamp(x) / 1000000))
df_readability_kickstarter_small['timestamp_deadline'] = df_readability_kickstarter_small['project_deadline'].apply(lambda x: (datetime.timestamp(x) / 1000000))

## Response Variable
CF_success = df_readability_kickstarter_small['project_pledged'].apply(lambda x: np.log(x) if x != 0 else 0)

## Explanatory Variables
readability = df_readability_kickstarter_small['flesch_kincaid']
readability_squared = df_readability_kickstarter_small['flesch_kincaid'] * df_readability_kickstarter_small['flesch_kincaid']

##Control Variables
loved_project = df_readability_kickstarter_small['project_staff_pick']
video = df_readability_kickstarter_small['project_video_url'].apply(lambda x: 0 if pd.isna(x) else 1)
funding_duration = df_readability_kickstarter_small['timestamp_deadline'] - df_readability_kickstarter_small['timestamp_launched']
number_of_words = df_readability_kickstarter_small['num_words']
log_funding_goal = df_readability_kickstarter_small['project_goal'].apply(lambda x: np.log(x))
updates = df_readability_kickstarter_small['updates_until_deadline']
rewards = df_readability_kickstarter_small['project_rewards']
number_projects = df_readability_kickstarter_small['project_creator_project_number'].apply(lambda x: 0 if pd.isna(x) else x)

In [ ]:
# Create a new DataFrame to store the variables
df_stochastic = pd.DataFrame({
    'Crowdfunding success': CF_success,
    'Readability': readability,
    'Readability^2': readability_squared,
    #'Time': time,
    #'Readability * Time': readabilityXtime,
    #'Readability^2 * Time': readability_squaredXtime,
    'Loved Project': loved_project,
    'Video': video,
    'Funding Duration': funding_duration,
    'Number of Words': number_of_words,
    'Log(Funding Goal)': log_funding_goal,
    'Updates': updates,
    'Rewards': rewards,
    'Number of projects': number_projects,
    'date': df_readability_kickstarter_small['project_launched_at']
})

In [ ]:
regressor_col = ['Readability', 'Readability^2', 'Loved Project', 'Video', 'Funding Duration', 'Number of Words',
                  'Log(Funding Goal)', 'Updates', 'Rewards', 'Number of projects']
response_col = 'Crowdfunding success'
date_col='date'

In [ ]:
ktr = KTR(
    response_col=response_col,
    date_col=date_col,
    regressor_col=regressor_col,
    prediction_percentiles=[2.5, 97.5],
    seed=2013,
    estimator='pyro-svi',
)

In [ ]:
ktr.fit(df=df_stochastic)
ktr.predict(df=df_stochastic, decompose=True).head(5)

In [ ]:
ktr.plot_regression_coefs(figsize=(20, 10), include_ci=True);